# Playground for Recommeder System slides

- Stephen W. Thomas
- Used for MMA 869, MMAI 869, and GMMA 869

In [1]:
import datetime
print(datetime.datetime.now())

2020-10-06 08:20:30.456308


In [2]:
import pandas as pd
pd.show_versions(as_json=False)


INSTALLED VERSIONS
------------------
commit: None
python: 3.6.4.final.0
python-bits: 64
OS: Windows
OS-release: 10
machine: AMD64
processor: Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
byteorder: little
LC_ALL: None
LANG: None
LOCALE: None.None

pandas: 0.24.2
pytest: 3.3.2
pip: 19.0.3
setuptools: 38.4.0
Cython: 0.27.3
numpy: 1.16.4
scipy: 1.2.1
pyarrow: None
xarray: None
IPython: 6.2.1
sphinx: 1.6.6
patsy: 0.5.0
dateutil: 2.6.1
pytz: 2017.3
blosc: None
bottleneck: 1.2.1
tables: 3.4.2
numexpr: 2.6.9
feather: None
matplotlib: 3.0.0
openpyxl: 2.4.10
xlrd: 1.1.0
xlwt: 1.3.0
xlsxwriter: 1.0.2
lxml.etree: 4.3.4
bs4: 4.6.0
html5lib: 1.0.1
sqlalchemy: 1.2.1
pymysql: None
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None
gcsfs: None


In [3]:
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Manual Examples

In [4]:
U = np.matrix([
    [3, 5, 4, np.nan, 0, 1], 
    [3, 5, np.nan, 3, 4, 0],
    [np.nan, 4, 5, np.nan, np.nan, 2],
    [4, np.nan, 3, 1, 2, 4],
    [5, 4, 4, np.nan, 4, 5],
    [0, 1, np.nan, 5, 4, 5]
]).T
U
U.shape

matrix([[ 3.,  3., nan,  4.,  5.,  0.],
        [ 5.,  5.,  4., nan,  4.,  1.],
        [ 4., nan,  5.,  3.,  4., nan],
        [nan,  3., nan,  1., nan,  5.],
        [ 0.,  4., nan,  2.,  4.,  4.],
        [ 1.,  0.,  2.,  4.,  5.,  5.]])

(6, 6)

In [5]:
from scipy.spatial import distance

u = U[1] # Mr Blue
u

for j in range(0, U.shape[1]):
    v = U[j]
    ind = np.isfinite(u) & np.isfinite(v)
    if np.any(ind) == False:
        cos_dist = 1
        cos_sim  = 1 - cos_dist
        euc_dist = 1000
        euc_sim  = 1/(1+euc_dist)
    else:
        #print(u[ind])
        #print(v[ind])
        cos_dist = distance.cosine(u[ind], v[ind])
        cos_sim  = 1 - cos_dist
        euc_dist = distance.euclidean(u[ind], v[ind])
        euc_sim  = 1/(1+euc_dist)
    print("{}, cos_sim={:.3f}, cos_dist={:.3f}, euc_sim={:.3f}, euc_dist={:.3f}".format(j, cos_sim, cos_dist, euc_sim, euc_dist))

matrix([[ 5.,  5.,  4., nan,  4.,  1.]])

0, cos_sim=0.932, cos_dist=0.068, euc_sim=0.240, euc_dist=3.162
1, cos_sim=1.000, cos_dist=0.000, euc_sim=1.000, euc_dist=0.000
2, cos_sim=0.982, cos_dist=0.018, euc_sim=0.414, euc_dist=1.414
3, cos_sim=0.673, cos_dist=0.327, euc_sim=0.183, euc_dist=4.472
4, cos_sim=0.705, cos_dist=0.295, euc_sim=0.145, euc_dist=5.916
5, cos_sim=0.562, cos_dist=0.438, euc_sim=0.113, euc_dist=7.874


In [6]:
u = U[:, 3] # LOTR

for j in range(0, U.shape[0]):
    v = U[:, j]
    ind = np.isfinite(u) & np.isfinite(v)
    if np.any(ind) == False:
        cos_dist = 1
        cos_sim  = 1 - cos_dist
        euc_dist = 1000
        euc_sim  = 1/(1+euc_dist)
    else:
        cos_dist = distance.cosine(u[ind], v[ind])
        cos_sim  = 1 - cos_dist
        euc_dist = distance.euclidean(u[ind], v[ind])
        euc_sim  = 1/(1+euc_dist)
    print("{}, cos_sim={:.3f}, cos_dist={:.3f}, euc_sim={:.3f}, euc_dist={:.3f}".format(j, cos_sim, cos_dist, euc_sim, euc_dist))

0, cos_sim=0.819, cos_dist=0.181, euc_sim=0.205, euc_dist=3.873
1, cos_sim=0.648, cos_dist=0.352, euc_sim=0.167, euc_dist=5.000
2, cos_sim=0.854, cos_dist=0.146, euc_sim=0.261, euc_dist=2.828
3, cos_sim=1.000, cos_dist=0.000, euc_sim=1.000, euc_dist=0.000
4, cos_sim=0.988, cos_dist=0.012, euc_sim=0.274, euc_dist=2.646
5, cos_sim=0.668, cos_dist=0.332, euc_sim=0.141, euc_dist=6.083


# Jaccard

In [7]:
def jaccard_similarity(l1, l2):
    intersection = len(list(set(l1).intersection(l2)))
    union = (len(l1) + len(l2)) - intersection
    return float(intersection / union)

list1 = ['dog', 'cat', 'fish']
list2 = ['dog', 'cat', 'horse', 'pig']
jaccard_similarity(list1, list2)

0.4

# Surprise Package

In [8]:
from surprise import SVD
from surprise import Dataset
from surprise import get_dataset_dir

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

In [10]:
import io  # needed because of weird encoding of u.item file
import itertools

def read_item_names():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid

# Read the mappings raw id <-> movie name
rid_to_name, name_to_rid = read_item_names()

# Look at 10 random elements in the dict
list(itertools.islice(name_to_rid.items(), 20))

[('Toy Story (1995)', '1'),
 ('GoldenEye (1995)', '2'),
 ('Four Rooms (1995)', '3'),
 ('Get Shorty (1995)', '4'),
 ('Copycat (1995)', '5'),
 ('Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', '6'),
 ('Twelve Monkeys (1995)', '7'),
 ('Babe (1995)', '8'),
 ('Dead Man Walking (1995)', '9'),
 ('Richard III (1995)', '10'),
 ('Seven (Se7en) (1995)', '11'),
 ('Usual Suspects, The (1995)', '12'),
 ('Mighty Aphrodite (1995)', '13'),
 ('Postino, Il (1994)', '14'),
 ("Mr. Holland's Opus (1995)", '15'),
 ('French Twist (Gazon maudit) (1995)', '16'),
 ('From Dusk Till Dawn (1996)', '17'),
 ('White Balloon, The (1995)', '18'),
 ("Antonia's Line (1995)", '19'),
 ('Angels and Insects (1995)', '20')]

In [11]:
type(data)

surprise.dataset.DatasetAutoFolds

In [12]:
type(data.raw_ratings)

# Format is:
# user item rating timestamp
data.raw_ratings[0]
data.raw_ratings[1]

list

('196', '242', 3.0, '881250949')

('186', '302', 3.0, '891717742')

In [13]:
df = pd.DataFrame(data.raw_ratings, columns=['user', 'item', 'rating', 'timestamp'])

In [14]:
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [15]:
item_names = pd.DataFrame.from_dict(rid_to_name, orient="index", columns=['item_name'])
item_names['item'] = item_names.index
item_names.head()
df = pd.merge(df, item_names)
df.head()

,item_name,item
1,Toy Story (1995),1
2,GoldenEye (1995),2
3,Four Rooms (1995),3
4,Get Shorty (1995),4
5,Copycat (1995),5


,user,item,rating,timestamp,item_name
0,196,242,3.0,881250949,Kolya (1996)
1,63,242,3.0,875747190,Kolya (1996)
2,226,242,5.0,883888671,Kolya (1996)
3,154,242,3.0,879138235,Kolya (1996)
4,306,242,5.0,876503793,Kolya (1996)


In [16]:
df[df['user'] == '22']

,user,item,rating,timestamp,item_name
414,22,377,1.0,878887116,Heavyweights (1994)
1044,22,265,3.0,878888066,"Hunt for Red October, The (1990)"
1180,22,451,4.0,878887062,Grease (1978)
1901,22,222,4.0,878887925,Star Trek: First Contact (1996)
2288,22,29,1.0,878888228,Batman Forever (1995)
2978,22,144,5.0,878887680,Die Hard (1988)
3141,22,118,4.0,878887983,Twister (1996)
3940,22,546,3.0,878888107,Broken Arrow (1996)
5653,22,194,5.0,878886607,"Sting, The (1973)"
6655,22,201,4.0,878886449,Evil Dead II (1987)


# Build the Recommender System

In [17]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, SVD

trainset, testset = train_test_split(data, test_size=.25)

In [18]:
sim_options = {'name': 'MSD',
               'user_based': True  # compute  similarities between items
               }

algo = KNNBasic(k=10, min_k=4, sim_options=sim_options)
%time algo.fit(trainset)
%time predictions = algo.test(testset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Wall time: 481 ms


Wall time: 2.74 s


In [19]:
accuracy.rmse(predictions)

RMSE: 0.9949


0.9948830508504914

In [20]:
predictions[0:10]

[Prediction(uid='49', iid='569', r_ui=3.0, est=1.597522731937372, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='294', iid='676', r_ui=3.0, est=3.4210954792447312, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='280', iid='576', r_ui=3.0, est=2.604068465422969, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='521', iid='249', r_ui=4.0, est=3.3534242538109615, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='294', iid='678', r_ui=2.0, est=2.9279124900539215, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='894', iid='300', r_ui=4.0, est=3.9433179415110753, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='197', iid='182', r_ui=3.0, est=4.424691912614842, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='181', iid='260', r_ui=1.0, est=2.1063734340434794, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='886', iid='801', r_ui=3.0, 

# Predict for a specific user/item pair

In [21]:
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, verbose=True)
pred

user: 196        item: 302        r_ui = None   est = 4.00   {'actual_k': 10, 'was_impossible': False}


Prediction(uid='196', iid='302', r_ui=None, est=4.0, details={'actual_k': 10, 'was_impossible': False})

# Get Top-Predicted Items for a Specific User

In [22]:
from collections import defaultdict

def get_top_n(algo, predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        #rid = algo.trainset.to_raw_iid(iid)
        item_name = rid_to_name[iid]
        #print('uid={}, iid={}, item_name={}, est={}'.format(uid, iid, item_name, est))
        top_n[uid].append((iid, item_name, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[2], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def get_user_neighbors(uid, k=10):
    # Retrieve inner id of the movie
    #raw_id = name_to_rid[item_name]
    #inner_id = algo.trainset.to_inner_iid(raw_id)

    # Retrieve inner ids of the nearest neighbors of the movie.
    neighbors = algo.get_neighbors(uid, k=10)

    # Convert inner ids of the neighbors into names.
    #neighbors = (algo.trainset.to_raw_iid(inner_id) for inner_id in neighbors)
    #neighbors = (rid_to_name[rid] for rid in neighbors)
    return neighbors

def user_info(uid, df, top_n, predictions):
    print('Everything we know about User {}'.format(uid))
    
    df_user = df[df['user'] == uid]
    print('Rating stats:')
    print(df_user['rating'].value_counts(sort=False).sort_index())
    
    print('\nTop 10 movies:')
    print(df_user.sort_values(by=['rating'], ascending=False).head(5))
    
    print('\nBottom 10 movies:')
    print(df_user.sort_values(by=['rating'], ascending=True).head(5))
    
    print('\nTop 10 predictions:')
    for iid, item_name, rating in top_n[uid]:
        print("{}, {}, {}".format(iid, item_name, rating))

In [23]:
top_n = get_top_n(algo, predictions, n=10)

In [24]:
user_info(str(653), df, top_n, predictions)

Everything we know about User 653
Rating stats:
1.0    50
2.0    82
3.0    77
4.0    53
5.0    21
Name: rating, dtype: int64

Top 10 movies:
      user item  rating  timestamp                 item_name
50926  653   50     5.0  878854100          Star Wars (1977)
46094  653  188     5.0  878854145  Full Metal Jacket (1987)
71080  653  180     5.0  878854593     Apocalypse Now (1979)
29817  653  127     5.0  878853780     Godfather, The (1972)
70088  653  239     5.0  878854475           Sneakers (1992)

Bottom 10 movies:
      user  item  rating  timestamp  \
78597  653   571     1.0  880153406   
92263  653   638     1.0  878866636   
47883  653   407     1.0  878867398   
90863  653  1183     1.0  880153329   
90500  653   136     1.0  880149965   

                                               item_name  
78597                            Another Stakeout (1993)  
92263  Return of Martin Guerre, The (Retour de Martin...  
47883                                    Spy Hard (1996)  
908

In [25]:
user_info(str(2), df, top_n, predictions)

Everything we know about User 2
Rating stats:
1.0     4
2.0     1
3.0    17
4.0    27
5.0    13
Name: rating, dtype: int64

Top 10 movies:
      user item  rating  timestamp                      item_name
31       2  242     5.0  888552084                   Kolya (1996)
84916    2  311     5.0  888552084  Wings of the Dove, The (1997)
66348    2  285     5.0  888552084          Secrets & Lies (1996)
74975    2  316     5.0  888979693      As Good As It Gets (1997)
72933    2  283     5.0  888552084                    Emma (1996)

Bottom 10 movies:
      user item  rating  timestamp  \
89687    2  314     1.0  888980085   
78824    2  309     1.0  888980029   
15258    2  294     1.0  888551648   
47094    2  315     1.0  888550774   
29561    2   10     2.0  888551853   

                                         item_name  
89687  3 Ninjas: High Noon At Mega Mountain (1998)  
78824                              Deceiver (1997)  
15258                             Liar Liar (1997)  
47094

In [26]:
user_info(str(8), df, top_n, predictions)

Everything we know about User 8
Rating stats:
1.0     5
2.0     4
3.0    10
4.0    19
5.0    21
Name: rating, dtype: int64

Top 10 movies:
      user item  rating  timestamp                            item_name
2138     8  222     5.0  879362356      Star Trek: First Contact (1996)
45967    8  188     5.0  879362356             Full Metal Jacket (1987)
28892    8  228     5.0  879362286  Star Trek: The Wrath of Khan (1982)
29676    8  127     5.0  879362123                Godfather, The (1972)
33226    8   56     5.0  879362183                  Pulp Fiction (1994)

Bottom 10 movies:
      user item  rating  timestamp                        item_name
29509    8  688     1.0  879361732        Leave It to Beaver (1997)
48523    8  259     1.0  879361604      George of the Jungle (1997)
81391    8  687     1.0  879361825             McHale's Navy (1997)
84270    8  457     1.0  879361825  Free Willy 3: The Rescue (1997)
19095    8  385     1.0  879362124                 True Lies (1994)

T

In [27]:
# Print the recommended items for all users
#for uid, user_ratings in top_n.items():
    #print(uid, [iid for (iid, _) in user_ratings])

# Get Nearest Neighbors of an Item

In [28]:
def get_item_neighbors(item_name, k=10):
    # Retrieve inner id of the movie
    raw_id = name_to_rid[item_name]
    inner_id = algo.trainset.to_inner_iid(raw_id)

    # Retrieve inner ids of the nearest neighbors of the movie.
    neighbors = algo.get_neighbors(inner_id, k=10)

    # Convert inner ids of the neighbors into names.
    neighbors = (algo.trainset.to_raw_iid(inner_id) for inner_id in neighbors)
    neighbors = (rid_to_name[rid] for rid in neighbors)
    return neighbors

item_name = 'Toy Story (1995)'
print('The nearest neighbors of {} are:'.format(item_name))
for movie in get_item_neighbors(item_name):
    print(movie)
    
    
item_name = 'Star Wars (1977)'
print('\nThe nearest neighbors of {} are:'.format(item_name))
for movie in get_item_neighbors(item_name):
    print(movie)

The nearest neighbors of Toy Story (1995) are:
To Catch a Thief (1955)
From Dusk Till Dawn (1996)
Devil's Advocate, The (1997)
Poison Ivy II (1995)
Aristocats, The (1970)
Welcome to the Dollhouse (1995)
Jungle Book, The (1994)
Nutty Professor, The (1996)
Apt Pupil (1998)
Fierce Creatures (1997)

The nearest neighbors of Star Wars (1977) are:
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)
American President, The (1995)
Rebel Without a Cause (1955)
Striptease (1996)
Pretty Woman (1990)
Waterworld (1995)
Field of Dreams (1989)
Rock, The (1996)
From Dusk Till Dawn (1996)
Desperate Measures (1998)


# Cross Validation

In [29]:
from surprise.model_selection import cross_validate 
algo = SVD()

# Run 5-fold cross-validation and print results.
%time cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9307  0.9303  0.9373  0.9410  0.9407  0.9360  0.0047  
MAE (testset)     0.7335  0.7338  0.7390  0.7435  0.7398  0.7379  0.0038  
Fit time          5.12    5.36    5.47    5.99    5.71    5.53    0.30    
Test time         0.22    0.14    0.14    0.35    0.15    0.20    0.08    
Wall time: 29.5 s


{'fit_time': (5.124032020568848,
  5.359018325805664,
  5.466701507568359,
  5.988003253936768,
  5.712029933929443),
 'test_mae': array([0.73349391, 0.73382195, 0.73904291, 0.74347112, 0.73984885]),
 'test_rmse': array([0.93069965, 0.93033751, 0.93726616, 0.94099154, 0.94067896]),
 'test_time': (0.21901488304138184,
  0.1379854679107666,
  0.1430377960205078,
  0.3480041027069092,
  0.14500713348388672)}

In [30]:
%time cross_validate(KNNBasic(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9886  0.9802  0.9672  0.9759  0.9836  0.9791  0.0073  
MAE (testset)     0.7790  0.7740  0.7642  0.7742  0.7752  0.7733  0.0049  
Fit time          0.65    0.63    0.54    0.55    0.56    0.59    0.04    
Test time         3.53    3.61    3.63    3.50    3.42    3.54    0.08    
Wall time: 21.5 s


{'fit_time': (0.653998851776123,
  0.6290009021759033,
  0.5449988842010498,
  0.552994966506958,
  0.5619478225708008),
 'test_mae': array([0.77899822, 0.77402762, 0.76423829, 0.77423802, 0.77520948]),
 'test_rmse': array([0.9886123 , 0.98020811, 0.96723332, 0.97589682, 0.98358926]),
 'test_time': (3.5259997844696045,
  3.607003927230835,
  3.6330010890960693,
  3.4990382194519043,
  3.423039674758911)}

# Grid Search

## SVD Algo

In [31]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

%time gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

Wall time: 49.5 s
0.964324110413269
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


# KNN Algo

In [32]:
param_grid_knn = {'k': [2, 10, 20],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 5, 20],
                              'user_based': [False, True],
                              'verbose': [False]}
              }

gs_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=3)
%time gs_knn.fit(data)
print(gs_knn.best_score['rmse'])
print(gs_knn.best_params['rmse'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

KeyboardInterrupt: 

AttributeError: 'GridSearchCV' object has no attribute 'best_score'

In [ ]:
gs_knn.cv_results

In [ ]:
param_grid_knn = {'k': [2, 10, 20],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 5, 20],
                              'user_based': [True],
                              'verbose': [False]}
              }


gs_knn = GridSearchCV(KNNWithMeans, param_grid_knn, measures=['rmse', 'mae'], cv=3)
%time gs_knn.fit(data)
print(gs_knn.best_score['rmse'])
print(gs_knn.best_params['rmse'])